## Data Science Academy

### Projeto e Implementação de Plataforma de Dados com Snowflake

### Lab 6

### Pipeline de Machine Learning com Snowpark no Snowflake

## Carregando Snowpark e Outros Pacotes

In [ ]:
# Imports
import os
import json
import pickle
import joblib
import optuna
import logging
import numpy as np
import pandas as pd
import snowflake.connector
from typing import List, Union, Tuple
from datetime import datetime, timedelta
from xgboost import XGBClassifier
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import (confusion_matrix,
                             accuracy_score,
                             precision_score,
                             recall_score,
                             f1_score,
                             roc_auc_score,
                             ConfusionMatrixDisplay)
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import call_udf, array_construct, pandas_udf, col, udf
from snowflake.snowpark import types as T
from snowflake.snowpark.context import get_active_session

## Criando a Sessão Snowpark

In [ ]:
# Conecta na sessão ativa do Snowpark
session = get_active_session()

In [ ]:
# Verifica os dados na conexão
session.sql("select current_warehouse(), current_database(), current_schema(), current_user(), current_role()").collect()

In [ ]:
# Nomes das áreas de stage
load_data_stage = "LOAD_DATA_STAGE"
model_stage = "MODEL_STAGE"
function_stage = "FUNCTION_STAGE"
package_stage = "PACKAGE_STAGE"

## Função Para Upload de Modelo Para o Stage

In [ ]:
# Função para upload do modelo para o stage
def dsa_upload_model_to_stage(session, obj, model_stage, model_id):
    
    # Verifica se o nome do stage já começa com '@', se não, adiciona
    if not model_stage.startswith("@"): 
        model_stage = f"@{model_stage}"
    
    # Define o caminho temporário para salvar o arquivo antes do upload
    temp_file_path = os.path.join("/tmp", model_id)
    
    # Salva o objeto fornecido como um arquivo temporário usando joblib
    joblib.dump(obj, temp_file_path)
    
    # Faz o upload do arquivo para o stage especificado
    session.file.put(temp_file_path, model_stage, overwrite = True, auto_compress = False)
    
    # Remove o arquivo temporário após o upload
    os.remove(temp_file_path)
    
    # Exibe uma mensagem indicando que o upload foi concluído com sucesso
    print(f"Upload do arquivo '{model_id}' finalizado com sucesso para o stage '{model_stage}'.")

## Função Para Deploy do Modelo Como UDF

In [ ]:
# Função de deploy
def dsa_deploy_model_as_udf(session: Session,
                            model,
                            scaler,
                            model_id: str,
                            model_stage: str,
                            function_stage: str,
                            required_packages: List[str]):
    
    # Define um identificador único para o scaler
    scaler_id = f"{model_id}_scaler"
    
    # Faz o upload do scaler para o stage especificado
    dsa_upload_model_to_stage(session, scaler, model_stage, scaler_id)
    
    # Faz o upload do modelo para o stage especificado
    dsa_upload_model_to_stage(session, model, model_stage, model_id)

    # Define a função de previsão que será registrada como UDF
    def predict(features: list) -> float:
        
        # Converte a lista de features para um array numpy e ajusta sua forma
        features_array = np.array(features).reshape(1, -1)
        
        # Aplica a transformação do scaler nos dados de entrada
        scaled_features = scaler.transform(features_array)
        
        # Retorna a previsão do modelo convertida para float
        return float(model.predict(scaled_features)[0])

    # Define o nome da UDF com base no ID do modelo
    udf_name = f"PREDICT_{model_id}"
    
    # Registra a UDF na sessão do Snowflake
    session.udf.register(func = predict,
                         name = udf_name,
                         stage_location = f"@{function_stage}",
                         is_permanent = True,
                         packages = required_packages,
                         imports = [f"@{model_stage}/{model_id}", f"@{model_stage}/{scaler_id}"])

    # Exibe uma mensagem de sucesso indicando o nome da UDF criada
    print(f"Deploy feito com sucesso para o UDF: {udf_name}")
    
    # Retorna o nome da UDF criada
    return udf_name

## Função Para Avaliar o Modelo

In [ ]:
# Função de avaliação do modelo
def dsa_evaluate_model(y_test, y_pred):

    # Cria a confusion matriz
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Extrai os valores
    TN, FP, FN, TP = conf_matrix.ravel()

    # Dicionário de métricas
    metrics = {"accuracy": accuracy_score(y_test, y_pred),
               "precision": precision_score(y_test, y_pred),
               "recall": recall_score(y_test, y_pred),
               "f1": f1_score(y_test, y_pred),
               "auc_roc": roc_auc_score(y_test, y_pred),
               "TN": TN,
               "FP": FP,
               "FN": FN,
               "TP": TP}

    return metrics

## Função Para Salvar Metadados de Treinamento do Modelo

In [ ]:
# Função para salvar metadados de treinamento
def dsa_save_training_info(session,
                           model_id,
                           model_name,
                           optimization,
                           training_table,
                           feature_columns,
                           metrics,
                           table_name = "MODEL_TRAINING_INFO"):
    
    # Importa a biblioteca datetime para registrar a data e hora do treinamento
    from datetime import datetime

    try:
        # Obtém a data e hora atuais no formato string
        training_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Converte a lista de colunas de features para uma string formatada corretamente
        feature_columns_str = ",".join([f"'{col}'" for col in feature_columns])

        # Define a consulta SQL para inserir os detalhes do treinamento no banco de dados
        insert_query = f"""
                INSERT INTO {table_name} (
                training_date,
                model_id,
                model_name,
                optimization,
                training_table,
                feature_columns,
                accuracy, precision, recall, f1_score, auc_roc,
                TN, FP, FN, TP
            )
            SELECT 
                '{training_date}', 
                '{model_id}', 
                '{model_name}', 
                {optimization}, 
                '{training_table}', 
                ARRAY_CONSTRUCT({feature_columns_str}),
                {metrics['accuracy']}, {metrics['precision']}, {metrics['recall']},
                {metrics['f1']}, {metrics['auc_roc']},
                {metrics['TN']}, {metrics['FP']}, {metrics['FN']}, {metrics['TP']};
            """

        # Executa a consulta SQL na sessão do banco de dados
        session.sql(insert_query).collect()
        
        # Exibe uma mensagem indicando que os detalhes do treinamento foram registrados com sucesso
        print(f"Metadados de treinamento do modelo '{model_id}' registrados com sucesso.")

    except Exception as e:
        # Captura e exibe erros que possam ocorrer durante a execução
        print(f"Erro de registo para o modelo '{model_id}': {e}")

## Função Para Treinamento e Deploy do Modelo

In [ ]:
# Função de treinamento e deploy
def dsa_train_and_deploy_model(session: Session,
                               model_name: str,
                               optimize: bool,
                               training_table: str) -> dict:

    # Imports
    import optuna
    from sklearn.model_selection import train_test_split, cross_val_score
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC
    from xgboost import XGBClassifier

    # Nomes dos modelos que serão criados
    model_abbreviations = {"Random Forest": "RF", "XGBoost": "XGB", "K-Nearest Neighbors": "KNN", "Support Vector Machine": "SVM"}

    # Pacotes necessários para o treinamento
    python_packages = ["pandas==2.2.3", "scikit-learn==1.5.2", "xgboost==1.7.3"]

    # Valor de sequência
    seq_value = str(session.sql("select MODEL_SEQ.nextval").collect()[0][0])

    # Obtém o nome do modelo
    model_reduced = model_abbreviations.get(model_name)

    # Id do modelo
    model_id = f"{model_reduced}_{seq_value}"

    # Id do scaler
    scaler_id = f"{model_reduced}_{seq_value}_scaler"

    # Define a tabela
    df = session.table(training_table).to_pandas()

    # Define x e y
    X = df.drop("TARGET", axis = 1)
    y = df["TARGET"]

    # Nomes dos atributos
    feature_columns = X.columns.to_numpy()

    # Divide os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

    # Cria o padronizador
    scaler = StandardScaler()

    # Fit e transform nos dados de treino
    X_train_scaled = scaler.fit_transform(X_train)

    # Somente em transform nos dados de teste
    X_test_scaled = scaler.transform(X_test)

    # Otimização
    if optimize:

        def objective(trial):

            # Inicializa a variável
            clf = None
            
            if model_reduced == "RF":

                # Hiperparâmetros para otimização do modelo
                params = {"max_depth": trial.suggest_int("max_depth", 2, 64),
                          "n_estimators": trial.suggest_int("n_estimators", 5, 100),
                          "max_samples": trial.suggest_float("rf_max_samples", 0.2, 1)}

                # Cria o modelo
                clf = RandomForestClassifier(**params, random_state = 42)

            elif model_reduced == "XGB":

                # Hiperparâmetros para otimização do modelo
                params = {"max_depth": trial.suggest_int("max_depth", 2, 20),
                          "n_estimators": trial.suggest_int("n_estimators", 10, 200),
                          "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3)}

                # Cria o modelo
                clf = XGBClassifier(**params, random_state=42)

            elif model_reduced == "KNN":

                # Hiperparâmetros para otimização do modelo
                params = {"n_neighbors": trial.suggest_int("n_neighbors", 1, 30)}

                # Cria o modelo
                clf = KNeighborsClassifier(**params)

            elif model_reduced == "SVM":

                # Hiperparâmetros para otimização do modelo
                params = {"C": trial.suggest_float("C", 1e-5, 1e5),
                          "gamma": trial.suggest_float("gamma", 1e-5, 1e2)}

                # Cria o modelo
                clf = SVC(**params, random_state = 42, probability = True)

            else:
                raise ValueError("Valor pode conter erro.")

            # Calcula o score
            score = cross_val_score(clf, X_train_scaled, y_train, cv = 3, scoring = "roc_auc").mean()

            # Extrai o melhor modelo
            trial.set_user_attr(key = "best_model", value = clf)

            return score

        # Função de callback
        def callback(study, trial):

            # Verifica se já temos o melhor modelo
            if study.best_trial.number == trial.number:
                study.set_user_attr(key = "best_model", value = trial.user_attrs["best_model"])

        # Cria o estudo de otimização
        study = optuna.create_study(direction = "maximize", sampler = optuna.samplers.RandomSampler(seed = 42))

        # Executa a otimização
        study.optimize(objective, n_trials = 100, callbacks = [callback])

        # Obtém o melhor resultado
        best_trial = study.best_trial

        # Melhor modelo do melhor resultado
        classifier = best_trial.user_attrs["best_model"]

    else:
        
        if model_reduced == "RF":
            classifier = RandomForestClassifier(random_state = 42)
        elif model_reduced == "XGB":
            classifier = XGBClassifier(random_state = 42)
        elif model_reduced == "KNN":
            classifier = KNeighborsClassifier(n_neighbors = 10)
        elif model_reduced == "SVM":
            classifier = SVC(random_state = 42)

    # Treina o modelo
    classifier.fit(X_train_scaled, y_train)

    # Faz a previsão
    y_pred = classifier.predict(X_test_scaled)

    # Avalia o modelo
    metrics = dsa_evaluate_model(y_test, y_pred)

    # Salva os metadados de treino
    dsa_save_training_info(session,
                           model_id,
                           model_name,
                           optimize,
                           training_table,
                           feature_columns,
                           metrics)

    # Deploy do UDF
    udf_name = dsa_deploy_model_as_udf(session,
                                       model = classifier,
                                       scaler = scaler,
                                       model_id = model_id,
                                       model_stage = "MODEL_STAGE",
                                       function_stage = "FUNCTION_STAGE",
                                       required_packages = python_packages)

    result = {"model_id": model_id, "udf_name": udf_name}

    return result

## Treinamento e Deploy da Versão 1 do Modelo

In [ ]:
# Treinamento e deploy da primeira versão do modelo
dsa_train_and_deploy_model(session, 
                           model_name = "XGBoost", 
                           optimize = True, 
                           training_table = "DATA_TABLE_1")

In [ ]:
# Lista o conteúdo do Model Stage
session.sql("ls @MODEL_STAGE").collect()

In [ ]:
# Converte a tabela em dataframe do pandas
df_training_info = session.table("MODEL_TRAINING_INFO").to_pandas()

In [ ]:
# Visualiza a tabela
df_training_info

## Treinamento e Deploy da Versão 2 do Modelo

In [ ]:
# Treinamento e deploy da segunda versão do modelo
dsa_train_and_deploy_model(session, 
                           model_name = "Random Forest", 
                           optimize = True, 
                           training_table = "DATA_TABLE_1")

## Treinamento e Deploy da Versão 3 do Modelo

In [ ]:
# Treinamento e deploy da terceira versão do modelo
dsa_train_and_deploy_model(session, 
                           model_name = "Support Vector Machine", 
                           optimize = True, 
                           training_table = "DATA_TABLE_1")

## Treinamento e Deploy da Versão 4 do Modelo

In [ ]:
# Treinamento e deploy da quarta versão do modelo
dsa_train_and_deploy_model(session, 
                           model_name = "K-Nearest Neighbors", 
                           optimize = True, 
                           training_table = "DATA_TABLE_1")

In [ ]:
# Lista o conteúdo do Model Stage
session.sql("ls @MODEL_STAGE").collect()

In [ ]:
# Converte a tabela em dataframe do pandas
df_training_info = session.table("MODEL_TRAINING_INFO").to_pandas()

In [ ]:
# Visualiza a tabela
df_training_info

Aqui termina o Lab 6 e então iniciamos o Lab 7.

## Data Science Academy

### Projeto e Implementação de Plataforma de Dados com Snowflake

### Lab 7

### Otimização de Modelo de Machine Learning com Optuna e Inferência com Snowpark

## Automatizando a Otimização de Hiperparâmetros com Optuna

In [ ]:
# Path para imports do Optuna
optuna_path = optuna.__path__[0]

# Registra a função como procedure
session.sproc.register(func = dsa_train_and_deploy_model,
                       name = "dsa_train_and_deploy_model",
                       packages = ["snowflake-snowpark-python",
                                   "scikit-learn==1.5.2",
                                   "xgboost==1.7.3",
                                   "sqlalchemy==1.4.39",
                                   "tqdm==4.64.1",
                                   "colorlog==5.0.1"],
                       imports = [optuna_path],
                       is_permanent = True,
                       stage_location = f"@{function_stage}",
                       replace = True)

In [ ]:
# Parâmetros (O XGBoost é o algoritmo que mais se beneficia da otimização de hiperparâmetros entre os algoritmos que estamos usando neste Lab)
model_name = "XGBoost"
optimization = True
training_table = "DATA_TABLE_2"

In [ ]:
# Executa a procedure
session.call("dsa_train_and_deploy_model",
             model_name,
             optimization,
             training_table)

In [ ]:
# Lista o conteúdo do Model Stage
session.sql("ls @MODEL_STAGE").collect()

In [ ]:
# Converte a tabela em dataframe do pandas
df_training_info = session.table("MODEL_TRAINING_INFO").to_pandas()

In [ ]:
# Visualiza a tabela
df_training_info

## Função Para Salvar Metadados da Inferência do Modelo

In [ ]:
# Função para salvar metadados da inferência
def dsa_save_inference_details(session,
                               model_id,
                               training_table,
                               test_table,
                               metrics,
                               table_name = "INFERENCE_RESULTS"):

    # Importa a classe datetime para manipular datas
    from datetime import datetime

    try:
        
        # Obtém a data e hora atual formatada
        inference_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Monta a query SQL para inserir os metadados da inferência na tabela especificada
        insert_query = f"""
                INSERT INTO {table_name} (
                    inference_date,
                    model_id,
                    training_table,
                    test_table,
                    accuracy, precision, recall, f1_score, auc_roc,
                    TN, FP, FN, TP
                )
                VALUES (
                    '{inference_date}',
                    '{model_id}',
                    '{training_table}',
                    '{test_table}',
                    {metrics['accuracy']}, {metrics['precision']}, {metrics['recall']},
                    {metrics['f1']}, {metrics['auc_roc']},
                    {metrics['TN']}, {metrics['FP']}, {metrics['FN']}, {metrics['TP']}
                );
                """

        # Executa a query no banco de dados
        session.sql(insert_query).collect()
        
        # Exibe mensagem de sucesso
        print(f"Metadados de inferencia salvos com sucesso para o modelo {model_id}.")

    except Exception as e:
        # Exibe mensagem de erro caso ocorra alguma exceção
        print(f"Erro ao salvar metadados de inferência: {e}")

## Função Para Inferência com o Modelo Treinado

In [ ]:
# Função de inferência
def dsa_run_inference(session: Session,
                      test_table: str,
                      model_id: str,
                      predictions_table: str = "PREDICTIONS_RESULT",
                      target_column: str = "TARGET") -> None:
    
    try:
        
        # Consulta informações do modelo treinado no banco de dados
        query_result = session.sql(
            f"""
            SELECT FEATURE_COLUMNS, TRAINING_TABLE
            FROM MODEL_TRAINING_INFO 
            WHERE MODEL_ID = '{model_id}'
            """
        ).collect()

        # Obtém a tabela usada para treinamento do modelo
        training_table = query_result[0]["TRAINING_TABLE"]
        
        # Obtém os nomes das colunas de características utilizadas no modelo
        feature_columns = query_result[0]["FEATURE_COLUMNS"]
        
        # Converte a string JSON contendo os nomes das colunas para uma lista
        feature_columns = json.loads(feature_columns)
        
        # Cria um array com as colunas de características para a inferência
        features_array = array_construct(*[col(c) for c in feature_columns])

        # Carrega os dados de teste na sessão
        df = session.table(test_table)
        
        # Verifica se a coluna alvo está presente nos dados de teste
        has_target = target_column in df.columns

        # Seleciona as colunas de características e a coluna alvo (se existir), 
        # aplicando a função de predição do modelo
        df_predictions = df.select(
            *feature_columns,
            *([target_column] if has_target else []),
            call_udf(f"PREDICT_{model_id}", features_array).alias("PREDICTION"),
        )

        # Salva os resultados das previsões na tabela de saída
        df_predictions.write.mode("overwrite").save_as_table(predictions_table)
        
        # Exibe uma mensagem informando que as previsões foram salvas com sucesso
        print(f"Resultados salvos na tabela {predictions_table}!")

        # Caso a coluna alvo esteja presente, calcula métricas de avaliação do modelo
        if has_target:
            
            # Converte os resultados das previsões para um DataFrame Pandas
            df_predictions_pd = df_predictions.to_pandas()
            
            # Calcula métricas de avaliação comparando previsões e valores reais
            metrics = dsa_evaluate_model(df_predictions_pd[target_column], 
                                         df_predictions_pd["PREDICTION"])

            # Salva os detalhes da inferência para futuras análises
            dsa_save_inference_details(session,
                                       model_id,
                                       training_table,
                                       test_table,
                                       metrics)

        return metrics

    except Exception as e:
        # Lança uma exceção em caso de erro durante a inferência
        raise Exception(f"Erro ao executar a inferência: {str(e)}")

## Executando a Inferência

In [ ]:
# Testando a inferência com um dos modelos
dsa_run_inference(session, test_table = "DATA_TABLE_2", model_id = "RF_2")

In [ ]:
# Registra a função como procedure para automação
session.sproc.register(func = dsa_run_inference,
                       name = "dsa_run_inference",
                       packages = ["snowflake-snowpark-python",
                                   "scikit-learn==1.2.1",
                                   "joblib==1.1.1",
                                   "xgboost==1.7.3"],
                       is_permanent = True,
                       stage_location = f"@{function_stage}",
                       replace = True)

In [ ]:
# Lista o conteúdo do Model Stage
session.sql("ls @MODEL_STAGE").collect()

In [ ]:
# Executa a procedure
session.call("dsa_run_inference", "DATA_TABLE_3", "XGB_5")

In [ ]:
# Executa a procedure
session.call("dsa_run_inference", "DATA_TABLE_3", "RF_2")

In [ ]:
# Executa a procedure
session.call("dsa_run_inference", "DATA_TABLE_3", "SVM_3")

In [ ]:
# Executa a procedure
session.call("dsa_run_inference", "DATA_TABLE_3", "KNN_4")

## Visualizando as Previsões

In [ ]:
# Converte para dataframe do pandas
df_predictions_result = session.table("PREDICTIONS_RESULT").to_pandas()

In [ ]:
# Visualiza a tabela de previsões
df_predictions_result

# Fim